In [1]:
import pandas as pd
import numpy as np

import sqlite3

names = ('Badges', 'PostLinks', 'Tags', 'Votes', 'Comments', 'Posts', 'Users')

for name in names:
    print(name)
    print(
        pd.read_csv(f"Exercise02/{name}.csv").head()
    )
    print('-'*10)

Badges
   Class                     Date  Id            Name  TagBased  UserId
0      3  2010-08-25T19:26:49.423   1  Autobiographer     False       2
1      3  2010-08-25T19:26:49.440   2  Autobiographer     False       3
2      3  2010-08-25T19:26:49.457   3  Autobiographer     False       4
3      3  2010-08-25T19:41:49.283   4  Autobiographer     False       6
4      3  2010-08-25T19:41:49.283   5  Autobiographer     False       7
----------
PostLinks
              CreationDate  Id  LinkTypeId  PostId  RelatedPostId
0  2010-08-26T20:33:50.797  31           1     262            261
1  2010-08-29T21:51:09.093  47           1     401            244
2  2010-08-30T01:57:19.137  48           1     132              3
3  2010-08-31T19:13:55.077  65           1     367             27
4  2010-08-31T19:13:55.077  66           1     367            246
----------
Tags
   Count  ExcerptPostId  Id    TagName  WikiPostId
0     21            NaN   1   schrader         NaN
1     44            NaN   

In [2]:
conn = sqlite3.connect('pandas_exercises')

for name in names:
    df = pd.read_csv(f"Exercise02/{name}.csv")
    df.to_sql(name, con=conn, if_exists='replace')

In [3]:
dfs = {
    name: pd.read_csv(f"Exercise02/{name}.csv")
    for name in names
}

In [4]:
query1 = """
--- Zadanie 1
SELECT PostId, COUNT(*) AS UpVotes
FROM Votes
WHERE VoteTypeId=2
GROUP BY PostId
"""

In [5]:
pd.read_sql_query(query1, con=conn)

,PostId,UpVotes
0,1,42
1,2,13
2,3,10
3,4,16
4,5,31
...,...,...
41776,63933,2
41777,63934,1
41778,63939,1
41779,63940,1


In [6]:
query2 = """
--- Zadanie 2
SELECT Title, Score, ViewCount, FavoriteCount
FROM Posts
WHERE PostTypeId=1 AND FavoriteCount >= 25 AND ViewCount>=10000
"""

In [7]:
query3 = """
--- Zadanie 3
SELECT Tags.TagName, Tags.Count, Posts.OwnerUserId,
Users.Location, Users.DisplayName
FROM Tags
JOIN Posts ON Posts.Id=Tags.WikiPostId
JOIN Users ON Users.AccountId=Posts.OwnerUserId
WHERE OwnerUserId != -1
ORDER BY Count DESC
"""

In [8]:
query4 = """
--- Zadanie 4
SELECT Posts.Title, RelatedTab.NumLinks FROM
(SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
FROM PostLinks GROUP BY RelatedPostId) AS RelatedTab
JOIN Posts ON RelatedTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY NumLinks DESC
"""

In [9]:
query5 = """
--- Zadanie 5
SELECT UpVotesTab.*, Posts.Title FROM
(
SELECT PostId, COUNT(*) AS UpVotes FROM Votes WHERE VoteTypeId=2 GROUP BY PostId
) AS UpVotesTab
JOIN Posts ON UpVotesTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY UpVotesTab.UpVotes DESC
"""

In [10]:
query6 = """
--- Zadanie 6
SELECT UpVotesTab.PostId, UpVotesTab.UpVotes, IFNULL(DownVotesTab.DownVotes, 0) AS DownVotes
FROM
(
SELECT PostId, COUNT(*) AS UpVotes FROM Votes
WHERE VoteTypeId=2 GROUP BY PostId
) AS UpVotesTab
LEFT JOIN
(SELECT PostId, COUNT(*) AS DownVotes FROM Votes
WHERE VoteTypeId=3 GROUP BY PostId
) AS DownVotesTab
ON UpVotesTab.PostId=DownVotesTab.PostId
"""

In [11]:
query7 = """
--- Zadanie 7
SELECT PostId, UpVotes-DownVotes AS Votes FROM (
SELECT UpVotesTab.PostId, UpVotesTab.UpVotes, IFNULL(DownVotesTab.DownVotes, 0) AS DownVotes
FROM
(
SELECT PostId, COUNT(*) AS UpVotes FROM Votes
WHERE VoteTypeId=2 GROUP BY PostId
) AS UpVotesTab
LEFT JOIN
(
SELECT PostId, COUNT(*) AS DownVotes
FROM Votes WHERE VoteTypeId=3 GROUP BY PostId
) AS DownVotesTab
ON UpVotesTab.PostId=DownVotesTab.PostId
UNION
SELECT DownVotesTab.PostId, IFNULL(UpVotesTab.UpVotes, 0) AS UpVotes, DownVotesTab.DownVotes
FROM
(
SELECT PostId, COUNT(*) AS DownVotes FROM Votes
WHERE VoteTypeId=3 GROUP BY PostId
) AS DownVotesTab
LEFT JOIN
(
SELECT PostId, COUNT(*) AS UpVotes FROM Votes
WHERE VoteTypeId=2 GROUP BY PostId
) AS UpVotesTab
ON DownVotesTab.PostId=UpVotesTab.PostId
)
"""